In [ ]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        images.append((filename, img))
    return images

def compute_hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=True)
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd, hog_image

def find_eye_template(template, image, threshold=0.6):
    # Вычисляем HOG для шаблона и изображения
    _, hog_template = compute_hog(template)
    _, hog_image = compute_hog(image)
    
    # Нормализация
    hog_template = cv2.normalize(hog_template, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    hog_image = cv2.normalize(hog_image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    res = cv2.matchTemplate(hog_image, hog_template, cv2.TM_CCOEFF_NORMED)  
    
    loc = np.where(res >= threshold)

    
    y_coords, x_coords = np.where(res >= threshold)
    
    # Если нет совпадений
    if len(x_coords) == 0:
        return (np.array([]), np.array([]))
    
    # Группируем близкие обнаружения
    points = np.column_stack((x_coords, y_coords))
    clusters = []
    min_dist = 100  
    
    while len(points) > 0:
        # Берем первую точку как центр кластера
        center = points[0]
        # Находим все точки в радиусе min_dist
        distances = np.sqrt(np.sum((points - center)**2, axis=1))
        cluster_indices = np.where(distances < min_dist)[0]
        cluster = points[cluster_indices]
        # Вычисляем среднее положение кластера
        mean_cluster = np.mean(cluster, axis=0).astype(int)
        clusters.append(mean_cluster)
        # Удаляем обработанные точки
        points = np.delete(points, cluster_indices, axis=0)

    if (len(cluster) > 2):
        find_eye_template(template, image, threshold=0.99)
    
    # Разделяем координаты обратно
    if clusters:
        clusters = np.array(clusters)
        return (clusters[:, 1], clusters[:, 0])  # Возвращаем (y, x) как в np.where
    else:
        return (np.array([]), np.array([]))
    
    

def process_images(template_path, images_folder, output_folder):
    
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    
    os.makedirs(output_folder, exist_ok=True)    
    
    images = load_images_from_folder(images_folder)
    
    for filename, img in images:
        # Находим шаблон на изображении
        loc = find_eye_template(template, img)
        
        # Будем рисовать прямоугольники вокруг найденных глаз
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        w, h = template.shape[::-1]
        
        for pt in zip(*loc[::-1]):
            cv2.rectangle(img_color, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
        
        # Результаты находятся в папке results
        output_path = os.path.join(output_folder, f"result_{filename}")
        cv2.imwrite(output_path, img_color)


template_path = "hog/eye.png"  # Путь к шаблону глаза
images_folder = "hog/images"   # Папка с изображениями
output_folder = "hog/results"  # Папка для результатов

process_images(template_path, images_folder, output_folder)
print("Обработка завершена. Результаты сохранены в", output_folder)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()